### FEW SHOT PROMPT EXAMPLE
Automatically derive the json from the schema, and use it as an example for the large language model to construct the next payload using prev conversations

In [6]:
import json
from jsonschema import Draft7Validator, exceptions
from faker import Faker
import random

fake = Faker()

def generate_example_json(schema):
    example_json = {}

    def generate_example_property(property_schema):
        if 'example' in property_schema:
            return property_schema['example']
        
        if 'type' in property_schema:
            if property_schema['type'] == 'object':
                example_property = {}
                if 'properties' in property_schema:
                    for prop_name, prop_schema in property_schema['properties'].items():
                        example_property[prop_name] = generate_example_property(prop_schema)
                return example_property
            elif property_schema['type'] == 'array':
                example_property = []
                if 'items' in property_schema:
                    for _ in range(2):  # Generate 2 items for the example array
                        example_property.append(generate_example_property(property_schema['items']))
                return example_property
            elif property_schema['type'] == 'string':
                if 'enum' in property_schema:
                    return random.choice(property_schema['enum'])  # Select a random enum value
                else:
                    return fake.word()
            elif property_schema['type'] == 'integer':
                return 0
            elif property_schema['type'] == 'boolean':
                return False

    for prop_name, prop_schema in schema['properties'].items():
        example_json[prop_name] = generate_example_property(prop_schema)

    return example_json

# Your schema
schema = {
    'required': ['name', 'photoUrls'],
    'type': 'object',
    'properties': {
        'id': {'type': 'integer', 'format': 'int64', 'example': 10},
        'name': {'type': 'string', 'example': 'doggie'},
        'category': {
            'type': 'object',
            'properties': {
                'id': {'type': 'integer', 'format': 'int64', 'example': 1},
                'name': {'type': 'string', 'example': 'Dogs'}
            },
            'xml': {'name': 'category'}
        },
        'photoUrls': {
            'type': 'array',
            'xml': {'wrapped': True},
            'items': {'type': 'string', 'xml': {'name': 'photoUrl'}}
        },
        'tags': {
            'type': 'array',
            'xml': {'wrapped': True},
            'items': {
                'type': 'object',
                'properties': {
                    'id': {'type': 'integer', 'format': 'int64'},
                    'name': {'type': 'string'}
                },
                'xml': {'name': 'tag'}
            }
        },
        'status': {
            'type': 'string',
            'description': 'pet status in the store',
            'enum': ['available', 'pending', 'sold']
        }
    },
    'xml': {'name': 'pet'}
}

# Generate an example JSON
example_json = generate_example_json(schema)

# Print the example JSON
print(json.dumps(example_json, indent=4))


{
    "id": 10,
    "name": "doggie",
    "category": {
        "id": 1,
        "name": "Dogs"
    },
    "photoUrls": [
        "season",
        "cup"
    ],
    "tags": [
        {
            "id": 0,
            "name": "second"
        },
        {
            "id": 0,
            "name": "billion"
        }
    ],
    "status": "pending"
}
